# Scraping by title

In [ ]:
import requests
import json
import pandas as pd
import time

#response = requests.get('https://www.googleapis.com/books/v1/volumes?q=inauthor:Caitlin Kelly')
url = 'https://www.googleapis.com/books/v1/volumes'
title_searchables = pd.read_csv('isbn13_1.csv')
df = pd.DataFrame(columns=['Author', 'Title', 'PublishedDate', 'ISBN13'])

i=0
n = title_searchables['title'].shape[0]
for titlesearch in title_searchables['title']:
    #print(f"Searching books for author {titlesearch}")
    params = {
        'q': f'intitle:"{titlesearch}"', 
        'startIndex': 0 
    }

    all_books = []


    while True:
        response = requests.get(url, params=params)
        data = json.loads(response.content)

        if 'items' in data:
            all_books.extend(data['items'])

        total_items = data.get('totalItems', 0)
        current_items = len(all_books)

        if current_items >= total_items:
            break

        params['startIndex'] = current_items
    #print("All books = ",len(all_books))
    for book in all_books:
        # Access book details using book['volumeInfo']
            #
            identifiers = book["volumeInfo"].get("industryIdentifiers", [])
            if 'authors' not in book["volumeInfo"] or 'publishedDate' not in book["volumeInfo"] or 'title' not in book["volumeInfo"]:
                continue
            title = book["volumeInfo"]["title"]
            author = book["volumeInfo"]["authors"]
            publishedDate = book["volumeInfo"]["publishedDate"]
            isbn13 = 'None'
            for identifier in identifiers:
                if identifier["type"] == "ISBN_13":
                    isbn13 = (identifier["identifier"])
                    break
            if isbn13 == 'None':
                continue
            new_data = ({'Title': title, 
                    'Author': author, 
                    'PublishedDate': publishedDate, 
                    'ISBN13': isbn13})
            new_row = pd.DataFrame(new_data)
            df = pd.concat([df, new_row], ignore_index=True)
            df = df.drop_duplicates(subset="ISBN13")
    i+=1
    if i%100 ==0:
        print(i)
        time.sleep(1)
        df.to_csv('All_isbn13_1.csv')